<a href="https://colab.research.google.com/github/Hrishikesh-Harsh/Text_Classification_IR/blob/main/Text_Classifier_IR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Use TF-IDF as prescribed
### Treat Abstract, Key and Title differently
### Order: $Key > Abstract ≈ Title$
### Hence, $W_k > W_a \approx W_t$
### For TF, we can use $TF = 1 + log(n_t*W_t + n_k*W_k + n_a*W_a)$
### One More Hyper-parameter to vary is Window size for (Word,Word) pairs
### Use $3$ different Window sizes for $Key, Abstract, Title$: $Win_k, Win_a, Win_t$

In [1]:
!git clone https://github.com/yao8839836/text_gcn.git

Cloning into 'text_gcn'...
remote: Enumerating objects: 26801, done.
remote: Total 26801 (delta 0), reused 0 (delta 0), pack-reused 26801
Receiving objects: 100% (26801/26801), 861.51 MiB | 17.25 MiB/s, done.
Resolving deltas: 100% (205/205), done.
Updating files: 100% (26397/26397), done.


In [ ]:
cd text_gcn

[Errno 2] No such file or directory: 'text_gcn'
/content


In [ ]:
!pip3 install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python3 remove_words.py 20ng

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{"shan't", 'aren', 'to', 'then', 'other', 'herself', 'ain', 'theirs', 'just', 'below', "needn't", 'why', 'on', 'haven', 'too', 'there', 'should', 'her', 'those', 've', 'no', 'nor', 'so', 'shan', 'what', 'and', 'when', 'once', 'me', 'themselves', 'out', 'itself', 'will', "it's", 'very', 'about', 'at', 'y', 'my', 'mustn', 'most', "shouldn't", 'hadn', 'who', "didn't", 'both', 'd', 'isn', 'where', 'is', 'against', "won't", 'i', 'here', 'myself', 'now', 'been', 'before', 'can', "you'd", 'as', 'in', 'we', "hadn't", 'while', 'do', 'not', 'into', 'but', 'm', 'did', 'same', 'ours', "you're", 'doesn', 'them', 'our', 'of', "she's", 'wasn', 'further', "aren't", 'you', "weren't", 'his', 'being', 'have', 'didn', 'they', 'their', 's', 'yourselves', "you've", 'it', 'each', 'during', 'having', 'all', 'its', 'be', 'your', 'she', 'any', "mightn't", 'he', 'above', "wouldn't", "you'll", 'down', 'a

In [ ]:
!python build_graph.py 20ng

[7532, 7533, 7534, 7535, 7536, 7537, 7538, 7539, 7540, 7541, 7542, 7543, 7544, 7545, 7546, 7547, 7548, 7549, 7550, 7551, 7552, 7553, 7554, 7555, 7556, 7557, 7558, 7559, 7560, 7561, 7562, 7563, 7564, 7565, 7566, 7567, 7568, 7569, 7570, 7571, 7572, 7573, 7574, 7575, 7576, 7577, 7578, 7579, 7580, 7581, 7582, 7583, 7584, 7585, 7586, 7587, 7588, 7589, 7590, 7591, 7592, 7593, 7594, 7595, 7596, 7597, 7598, 7599, 7600, 7601, 7602, 7603, 7604, 7605, 7606, 7607, 7608, 7609, 7610, 7611, 7612, 7613, 7614, 7615, 7616, 7617, 7618, 7619, 7620, 7621, 7622, 7623, 7624, 7625, 7626, 7627, 7628, 7629, 7630, 7631, 7632, 7633, 7634, 7635, 7636, 7637, 7638, 7639, 7640, 7641, 7642, 7643, 7644, 7645, 7646, 7647, 7648, 7649, 7650, 7651, 7652, 7653, 7654, 7655, 7656, 7657, 7658, 7659, 7660, 7661, 7662, 7663, 7664, 7665, 7666, 7667, 7668, 7669, 7670, 7671, 7672, 7673, 7674, 7675, 7676, 7677, 7678, 7679, 7680, 7681, 7682, 7683, 7684, 7685, 7686, 7687, 7688, 7689, 7690, 7691, 7692, 7693, 7694, 7695, 7696, 7697, 769

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
import math
import nltk
import re
import numpy as np
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Loading original .csv file

file = open('/content/drive/MyDrive/IR_Project/dataset/PubMed.csv',encoding='Latin1')
type(file)
csvreader = csv.reader(file)

In [ ]:
# Set of stopwords to be removed
stop_words = set(stopwords.words('english'))

In [ ]:
s = "Hello, this is Jay's brother."
l = word_tokenize(s)
print(l)

['Hello', ',', 'this', 'is', 'Jay', "'s", 'brother', '.']


In [ ]:
# Just the first line of the .csv file (Column Names)
header = []
header = next(csvreader)

In [ ]:
# vocab_words would be a set of all distinct words found in the dataset/.csv file minus the stopwords
vocab_words = set()
vocab_words_list = []

In [ ]:
rows = []
index = 0
title = []
keyword = []
abstract = []
for r in csvreader: 
  r_sub = []
  r[2] = r[2].replace('.',' ')
  r[2] = r[2].replace(',',' ')
  r[2] = r[2].replace(';',' ')
  r[2] = r[2].replace('|',' ')
  r[2] = r[2].replace('<',' ')
  r[2] = r[2].replace('>',' ')
  r[2] = r[2].replace(':',' ')
  r[2] = r[2].replace('=',' ')
  r[2] = r[2].replace('(',' ')
  r[2] = r[2].replace(')',' ')
  r[2] = r[2].replace('[',' ')
  r[2] = r[2].replace(']',' ')
  r[2] = r[2].replace('?',' ')


  r[3] = r[3].replace('.',' ')
  r[3] = r[3].replace(',',' ')
  r[3] = r[3].replace(';',' ')
  r[3] = r[3].replace('|',' ')
  r[3] = r[3].replace('<',' ')
  r[3] = r[3].replace('>',' ')
  r[3] = r[3].replace(':',' ')
  r[3] = r[3].replace('=',' ')
  r[3] = r[3].replace('(',' ')
  r[3] = r[3].replace(')',' ')
  r[3] = r[3].replace('[',' ')
  r[3] = r[3].replace(']',' ')
  r[3] = r[3].replace('?',' ')

  r[4] = r[4].replace('.',' ')
  r[4] = r[4].replace(',',' ')
  r[4] = r[4].replace(';',' ')
  r[4] = r[4].replace('|',' ')
  r[4] = r[4].replace('<',' ')
  r[4] = r[4].replace('>',' ')
  r[4] = r[4].replace(':',' ')
  r[4] = r[4].replace('=',' ')
  r[4] = r[4].replace('(',' ')
  r[4] = r[4].replace(')',' ')
  r[4] = r[4].replace('[',' ')
  r[4] = r[4].replace(']',' ')
  r[4] = r[4].replace('?',' ')

  words_title = word_tokenize(r[2])     # Tokenize the Title of that doc (row)
  words_keyword = word_tokenize(r[3])   # Tokenize the Keywords of that doc (row)
  words_abstract = word_tokenize(r[4])  # Tokenize the Abstract of that doc (row)

  title.append([])
  keyword.append([])
  abstract.append([])

  w_t = ""
  w_k = ""
  w_a = ""

  ''' 
      - Adding all non-stop-words to the vocabulary
      - Also maintaining doc-wise collection of Keywords, Title and Abstract words 
  '''

  for w in words_title:
    if w not in stop_words and w!="'s":
        w_t = w_t + w + " "
        vocab_words.add(w)
        # vocab_words_list.append(w)
        title[index].append(w)
  
  for w in words_keyword:
    if w not in stop_words and w!="'s":
        w_k = w_k + w + " "
        vocab_words.add(w)
        # vocab_words_list.append(w)
        keyword[index].append(w)

  for w in words_abstract:
    if w not in stop_words and w!="'s":
        w_a = w_a + w + " "
        vocab_words.add(w)
        # vocab_words_list.append(w)
        abstract[index].append(w)
  
  index=index+1

  '''
    - Creating 'rows' to write back to Clean File
  '''
  for i in range(0,len(r)):
    if(i==2):
      r_sub.append(w_t)
    elif(i==3):
      r_sub.append(w_k)
    elif(i==4):
      r_sub.append(w_a) 
    else:
      r_sub.append(r[i])
  
  rows.append(r_sub)


In [ ]:
for wd in vocab_words:
  vocab_words_list.append(wd)

In [ ]:
# Globals
docs_size = len(rows)
vocab_size = len(vocab_words)
wt_k = 3    # Weight to be given to Keywords in tf score
wt_t = 1    # Weight to be given to Title in tf score
wt_a = 1    # Weight to be given to Abstract in tf score

In [ ]:
'''
  To write back to Clean File
'''
file = open('/content/drive/MyDrive/IR_Project/dataset/PubMed_Clean.csv', 'w', newline='')
writer = csv.writer(file)
writer.writerow(header)

for r in rows:
  writer.writerow(r)


In [ ]:
'''
  Create a dict to store {word: (ID, idf)} mapping
'''
dict_vocab = {}

for i,w in enumerate(vocab_words):
  dict_vocab[w]=(i,0)

In [ ]:
for i,w in enumerate(vocab_words):
  if(i>5):
    break
  print(w)

notwithstanding
PICS
landscape
NICE
Cups
PPV


In [ ]:
'''
  Find idf values of all words and write them to idf.csv file
  to save time of calculating the values everytime
'''
file = open('/content/drive/MyDrive/IR_Project/dataset/idf.csv', 'w', newline='') 
writer = csv.writer(file) 
writer.writerow(["id","idf"])
for wd in dict_vocab: 
  count = 0
  for i,doc in enumerate(rows):
    flag = 0 
    for w_t in title[i]:
      if(wd==w_t):
        count=count+1
        flag = 1
        break 

    if(flag==1):
      continue 

    for w_k in keyword[i]:
      if(wd==w_k):
        count=count+1
        flag = 1
        break

    if(flag==1):
      continue 

    for w_a in abstract[i]:
      if(wd==w_a):
        count=count+1
        flag = 1
        break

    if(flag==1):
      continue

  (id,idf) = dict_vocab[wd]
  if(count==0):
    idf = 0
  else:
    idf = math.log((docs_size/count),10) 
  dict_vocab[wd] = (id,idf)
  templine=[]
  templine.append(id)
  templine.append(idf)
  writer.writerow(templine)
  print(templine)
  count = 0
writer.writerow("fine")
file.close()

In [ ]:
file = open('/content/drive/MyDrive/IR_Project/dataset/idf (1).csv',encoding='Latin1')
type(file)
csvreader = csv.reader(file)

header = []
header = next(csvreader)

In [ ]:
for val in dict_vocab: 
  r = next(csvreader)
  (id,idf) = dict_vocab[val]
  idf = float(r[1])
  dict_vocab[val] = (id,idf)

In [ ]:
for i,ele in enumerate(dict_vocab):
  if(i==5):
    break;
  
  print(ele,",",dict_vocab[ele])

notwithstanding , (0, 3.645078235)
PICS , (1, 2.832164878)
landscape , (2, 3.645078235)
NICE , (3, 3.94610823)
Cups , (4, 3.94610823)


In [ ]:
Adj_Matrix = np.zeros((docs_size+vocab_size,docs_size+vocab_size))

In [ ]:
for i,doc in enumerate(rows):
  for w_k in keyword[i]:
    Adj_Matrix[i,dict_vocab[w_k][0]+docs_size] += wt_k
    Adj_Matrix[dict_vocab[w_k][0]+docs_size,i] += wt_k

  for w_t in title[i]:
    Adj_Matrix[i,dict_vocab[w_t][0]+docs_size] += wt_t
    Adj_Matrix[dict_vocab[w_t][0]+docs_size,i] += wt_t

  for w_a in abstract[i]:
    Adj_Matrix[i,dict_vocab[w_a][0]+docs_size] += wt_a
    Adj_Matrix[dict_vocab[w_a][0]+docs_size,i] += wt_a


['Assessing', 'impacts', 'COVID-19', 'vaccination', 'programme']

In [ ]:
for i in range(0,Adj_Matrix.shape[0]):
  for j in range(0,Adj_Matrix.shape[0]):
    if(Adj_Matrix[i,j]!=0):
      if(i<j):
        wd = vocab_words_list[j-docs_size]
        idf = dict_vocab[wd][1]
        Adj_Matrix[i,j] = (1+math.log(Adj_Matrix[i,j],10))*idf
        print(wd,"|",idf,"|",Adj_Matrix[i,j])
      else:
        wd = vocab_words_list[i-docs_size]
        idf = dict_vocab[wd][1]
        Adj_Matrix[i,j] = (1+math.log(Adj_Matrix[i,j],10))*idf
        print(wd,"|",idf,"|",Adj_Matrix[i,j])

In [ ]:
# word co-occurence with context windows
window_size = 20
windows = []

for row in rows:
    content=row[2]+" "+row[3]+" "+row[4]
    words = content.split()
    length = len(words)
    if length <= window_size: 
        windows.append(words)
    else:
        # print(length, length - window_size + 1)
        for j in range(length - window_size + 1):
            window = words[j: j + window_size]
            windows.append(window)
            # print(window)

#calculating p(i) , word_window_freq has the number of windows a particular word appears in across all windows.
word_window_freq = {}
for window in windows:
    appeared = set()
    for i in range(len(window)):
        if window[i] in appeared:
            continue
        if window[i] in word_window_freq:
            word_window_freq[window[i]] += 1
        else:
            word_window_freq[window[i]] = 1
        appeared.add(window[i])


word_pair_count = {}
for window in windows:
    for i in range(1, len(window)):
        for j in range(0, i):
            word_i = window[i] #ith word in window
            word_i_id = dict_vocab[word_i][0]
            word_j = window[j] #jth word in range 0-i in the same window
            word_j_id = dict_vocab[word_j][0]
            if word_i_id == word_j_id:
                continue
            word_pair_str = str(word_i_id) + ',' + str(word_j_id) #concat id and use it to count a pair or p(i,j)
            if word_pair_str in word_pair_count: #word_pair_count stores number of pairs along with number of times they appear.
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1
            # two orders
            word_pair_str = str(word_j_id) + ',' + str(word_i_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1

row = []
col = []
weight = []

# pmi as weights

num_window = len(windows)

for key in word_pair_count:
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    count = word_pair_count[key] #p(i,j)
    word_freq_i = word_window_freq[vocab_words_list[i]] #p(i)
    word_freq_j = word_window_freq[vocab_words_list[j]] #p(j)
    pmi = math.log((1.0 * count / num_window) /(1.0 * word_freq_i * word_freq_j/(num_window * num_window)),10) #adj(i,j)
    if pmi <= 0:
        continue
    Adj_Matrix[docs_size+i][docs_size+i]=pmi


59620


In [ ]:
for i in range(Adj_Matrix.shape[0]):
  for j in range(Adj_Matrix.shape[1]):
    if(i==j):
      Adj_Matrix=1